In [1]:
#using Pkg
#Pkg.add("Gumbo")
#Pkg.add("Cascadia")
#Pkg.add("JSON")

In [2]:
using HTTP
using Gumbo
using Cascadia
using Dates
using JSON

In [3]:
# Yahoo!ニュースでのカテゴリー
topics_tags = ["domestic", "world", "business", "entertainment", "sports", "it", "science", "local"]

8-element Vector{String}:
 "domestic"
 "world"
 "business"
 "entertainment"
 "sports"
 "it"
 "science"
 "local"

In [4]:
base_url = "https://news.yahoo.co.jp/topics/"

"https://news.yahoo.co.jp/topics/"

In [5]:
# トピックス一覧から、タイトルとURL（概要）、投稿日時を取得
news_list_p = []
for category in topics_tags
    topics_url = base_url * category

    # 指定トピックのニュース一覧を取得
    response = HTTP.request("GET", topics_url)
    code = String(response.body)
    doc = Gumbo.parsehtml(code)
    #head = doc.root[1]
    body = doc.root[2]
    items = eachmatch(Selector(".newsFeed_item"), body) ;
    for item in items
        # url 
        link_items = eachmatch(Selector(".newsFeed_item_link"), item)
        if length(link_items) == 0   # 広告をスキップする
            continue
        end
        url = getattr(link_items[1], "href", nothing)
        # title
        title = nodeText(eachmatch(Selector(".newsFeed_item_title"), item)[1])
        # datetime
        dt = nodeText(eachmatch(Selector("time"), item)[1])
        result = Dict("url" => url, "title" => title, "datetime" => dt, "category" => category)
        push!(news_list_p, result)
    end
end

In [6]:
news_list_p

198-element Vector{Any}:
 Dict("datetime" => "6/29(水) 12:14", "title" => "東電管内「3日間で一番厳しい」", "url" => "https://news.yahoo.co.jp/pickup/6430987", "category" => "domestic")
 Dict("datetime" => "6/29(水) 12:03", "title" => "午前から猛暑日 関東で40℃予想も", "url" => "https://news.yahoo.co.jp/pickup/6430986", "category" => "domestic")
 Dict("datetime" => "6/29(水) 10:23", "title" => "日韓首脳、初めて対面しあいさつ", "url" => "https://news.yahoo.co.jp/pickup/6430971", "category" => "domestic")
 Dict("datetime" => "6/29(水) 9:50", "title" => "自民派閥の代理戦争 2人に明暗", "url" => "https://news.yahoo.co.jp/pickup/6430969", "category" => "domestic")
 Dict("datetime" => "6/29(水) 9:18", "title" => "猛暑で電力ひっ迫 照明等で節電を", "url" => "https://news.yahoo.co.jp/pickup/6430966", "category" => "domestic")
 Dict("datetime" => "6/29(水) 6:31", "title" => "関東内陸で40度か 北海道は大雨も", "url" => "https://news.yahoo.co.jp/pickup/6430948", "category" => "domestic")
 Dict("datetime" => "6/28(火) 22:13", "title" => "29日は広く猛暑日 40℃予想の所も", "url" => "https://news.yahoo.co.j

In [7]:
# URL（概要）のページから、URL（全文）を取得
news_list_d = []
# コラムなどは書き方が異なるので除外する
effective_url_base = r"https://news.yahoo.co.jp/articles/"
for ns in news_list_p
    response = HTTP.request("GET", ns["url"])
    ns_code = String(response.body)
    ns_doc = Gumbo.parsehtml(ns_code)
    body = ns_doc.root[2]
    detail_url = getattr(eachmatch(Selector("a[data-ual-gotocontent=\"true\"]"), body)[1], "href", nothing)
    if occursin(effective_url_base, detail_url)
        ns["url"] = detail_url
        push!(news_list_d, ns)
    end
end

LoadError: BoundsError: attempt to access 0-element Vector{HTMLNode} at index [1]

In [8]:
news_list_d

182-element Vector{Any}:
 Dict("datetime" => "6/29(水) 12:14", "title" => "東電管内「3日間で一番厳しい」", "url" => "https://news.yahoo.co.jp/articles/4a6765366d7f53abb6c22f6b358ce1966eadc1dd", "category" => "domestic")
 Dict("datetime" => "6/29(水) 12:03", "title" => "午前から猛暑日 関東で40℃予想も", "url" => "https://news.yahoo.co.jp/articles/c9a5e3fd1c4d8ffa1644e30cc691edd3da756540", "category" => "domestic")
 Dict("datetime" => "6/29(水) 10:23", "title" => "日韓首脳、初めて対面しあいさつ", "url" => "https://news.yahoo.co.jp/articles/5703e39ba38705c9a91de5df90fdbac72b1d08d1", "category" => "domestic")
 Dict("datetime" => "6/29(水) 9:50", "title" => "自民派閥の代理戦争 2人に明暗", "url" => "https://news.yahoo.co.jp/articles/296fa4fba39db8fc68991a687725325c861eafe9", "category" => "domestic")
 Dict("datetime" => "6/29(水) 9:18", "title" => "猛暑で電力ひっ迫 照明等で節電を", "url" => "https://news.yahoo.co.jp/articles/96e52a94fe4e54d83e701a46bf496f3af0232f0c", "category" => "domestic")
 Dict("datetime" => "6/29(水) 6:31", "title" => "関東内陸で40度か 北海道は大雨も", "url" 

In [9]:
# URL（本文）のページから、本文と、ニュースソースを取得
news_list = []
for ns in news_list_d
    response = HTTP.request("GET", ns["url"])
    code = String(response.body)
    doc = Gumbo.parsehtml(code)
    body = doc.root[2]
    source_parts = eachmatch(Selector("article > div.article_body > p"), body)
    source = length(source_parts) != 0 ? nodeText(source_parts[1]) : ""
    detail = nodeText(eachmatch(Selector("article > div.article_body > div> p"), body)[1])
    ns["detail"] = detail
    ns["source"] = source
    push!(news_list, ns)
end

In [10]:
news_list

182-element Vector{Any}:
 Dict("datetime" => "6/29(水) 12:14", "source" => "", "detail" => "29日も厳しい暑さが予想され、政府は東京電力管内で「電力需給ひっ迫注意報」を継続しています。この3日間で一番厳しい見通しだとしています。\n\n【図解】家庭でできる「無理の無い」節電メニュー\n\n政府は、29日も東京電力管内で、電力の余力を示す「予備率」が5％を下回る見通しだとして、「電力需給ひっ迫注意報」を継続しています。\n\n厳しい時間帯が午後3時から午後8時の間とこれまでで最も長く、予備率が安定供給が困難となる3％を下回る見通しの時間帯もあることから、この3日間で最も厳しい見通しとなっています。\n\n経産省は、冷房などを活用して熱中症対策に注意しつつ、使っていない照明を消すなど、できる限りの節電を呼びかけています。\n\n一方、30日運転を再開する千葉県にある姉崎火力発電所は、その準備段階としてすでにボイラーを点火し起動したということで、作業が順調に進めば、30日午前中には東京電力管内に電力供給を始める予定です。", "title" => "東電管内「3日間で一番厳しい」", "url" => "https://news.yahoo.co.jp/articles/4a6765366d7f53abb6c22f6b358ce1966eadc1dd", "category" => "domestic")
 Dict("datetime" => "6/29(水) 12:03", "source" => "ウェザーニュース", "detail" => "今日29日(水)も気温の上昇が早く、富山市など午前中から猛暑日のところが多くなっています。東京も33℃を突破しました。午後は40℃超えの危険な暑さになるところもあり、意識的な水分補給や塩分補給など熱中症対策が欠かせません。", "title" => "午前から猛暑日 関東で40℃予想も", "url" => "https://news.yahoo.co.jp/articles/c9a5e3fd1c4d8ffa1644e30cc691edd3da756540", "category" => "domestic")
 Dict("datet

In [11]:
# 保存
filename = "yahoo" * Dates.format(now(), "yyyymmdd") * ".txt"
open(filename, "w") do f
    println(f, json(news_list))
end

In [12]:
# 読み込み
data = JSON.parsefile("yahoo20220629.txt")

182-element Vector{Any}:
 Dict{String, Any}("datetime" => "6/29(水) 12:14", "source" => "", "detail" => "29日も厳しい暑さが予想され、政府は東京電力管内で「電力需給ひっ迫注意報」を継続しています。この3日間で一番厳しい見通しだとしています。\n\n【図解】家庭でできる「無理の無い」節電メニュー\n\n政府は、29日も東京電力管内で、電力の余力を示す「予備率」が5％を下回る見通しだとして、「電力需給ひっ迫注意報」を継続しています。\n\n厳しい時間帯が午後3時から午後8時の間とこれまでで最も長く、予備率が安定供給が困難となる3％を下回る見通しの時間帯もあることから、この3日間で最も厳しい見通しとなっています。\n\n経産省は、冷房などを活用して熱中症対策に注意しつつ、使っていない照明を消すなど、できる限りの節電を呼びかけています。\n\n一方、30日運転を再開する千葉県にある姉崎火力発電所は、その準備段階としてすでにボイラーを点火し起動したということで、作業が順調に進めば、30日午前中には東京電力管内に電力供給を始める予定です。", "title" => "東電管内「3日間で一番厳しい」", "url" => "https://news.yahoo.co.jp/articles/4a6765366d7f53abb6c22f6b358ce1966eadc1dd", "category" => "domestic")
 Dict{String, Any}("datetime" => "6/29(水) 12:03", "source" => "ウェザーニュース", "detail" => "今日29日(水)も気温の上昇が早く、富山市など午前中から猛暑日のところが多くなっています。東京も33℃を突破しました。午後は40℃超えの危険な暑さになるところもあり、意識的な水分補給や塩分補給など熱中症対策が欠かせません。", "title" => "午前から猛暑日 関東で40℃予想も", "url" => "https://news.yahoo.co.jp/articles/c9a5e3fd1c4d8ffa1644e30cc691edd3da756540", "category" =